# TF-TRT Inference

This notebook gives a high-level description of how TF-TRT optimizes graphs, and also, how to code this optimization. In the next notebook you will use this knowledge to code a TF-TRT optimization.

## Objectives

By the time you complete this notebook you will be able to:

- Describe how TF-TRT performs graph optimization
- Describe how to use `TrtGraphConverterV2` to code TF-TRT graph optimizations

## Network Transformation

TF-TRT performs several important transformations and optimizations to the neural network graph. First, layers with unused outputs are eliminated to avoid unnecessary computation. Next, where possible, convolution, bias, and ReLU layers are fused to form a single layer. *Figure 1* shows a typical convolutional network before optimization:

<div align="center">
    <img width="700px" src='images/network_optimization.png' />
    <p style="text-align: center;color:gray">Figure 1: An example convolutional  model with multiple convolutional and activation layers before optimization</p>
</div>

*Figure 2* shows the result of this vertical layer fusion on the original network from Figure 1 (fused layers are labeled CBR). Layer fusion improves the efficiency of running TF-TRT networks on the GPU.

<div align="center">
    <img width="600px" src='images/network_vertical_fusion.png' />
    <p style="text-align: center;color:gray">Figure 2: Fusing blocks into a single layer</p>
</div>

Another transformation is horizontal layer fusion, or layer aggregation, along with the required division of aggregated layers to their respective outputs, as Figure 3 shows.

Horizontal layer fusion improves performance by combining layers that take the same source tensor and apply the same operations with similar parameters, resulting in a single larger layer for higher computational efficiency. The example in Figure 3 shows the combination of 3 1×1 CBR layers from Figure 2 that take the same input into a single larger 1×1 CBR layer. Note that the output of this layer must be disaggregated to feed into the different subsequent layers from the original input graph.

<div align="center">
    <img width="600px" src='images/network_horizontal_fusion.png' />
    <p style="text-align: center;color:gray"> Figure 3. Horizontal layer fusion </p>
</div>

When optimizing a TensorFlow model, TF-TRT can optimize either a subgraph or the entire graph definition. This capability allows the optimization procedure to be applied to the graph where possible and skip the non-supported graph segments. As a result, if the existing model contains a non-supported layer or operation, TensorFlow can still optimize the graph. 

Please see the [TF-TRT User Guide](https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/index.html#supported-ops) for a full list of supported operators.

## TF-TRT Workflow

Below, you can see a typical workflow of TF-TRT:

![inference process](./images/inference_process_fp.png)

We now turn to the syntax for this one additional *Convert to TF-TRT* step.

## Graph Conversion

To perform graph conversion, we use `TrtGraphConverterV2`, passing it the directory of a saved model, and any updates we wish to make to its conversion parameters.

```python
from tensorflow.python.compiler.tensorrt import trt_convert as trt

trt.TrtGraphConverterV2(
    input_saved_model_dir=None,
    conversion_params=TrtConversionParams(precision_mode='FP32',
                                          max_batch_size=1
                                          minimum_segment_size=3,
                                          max_workspace_size_bytes=1073741824,
                                          use_calibration=True,
                                          maximum_cached_engines=1,
                                          is_dynamic_op=True,
                                          rewriter_config_template=None,
                                         )
```

### Conversion Parameters

Here is additional information about the most frequently adjusted conversion parameters, all of which you will have an opportunity to code with in later exercises.

* __precision_mode__: This parameter sets the precision mode; which can be one of FP32, FP16, or INT8. Precision lower than FP32, meaning FP16 and INT8, would improve the performance of inference. The FP16 mode uses Tensor Cores or half precision hardware instructions, if possible. The INT8 precision mode uses integer hardware instructions.

* __max_batch_size__: This parameter is the maximum batch size for which TF-TRT will optimize. At runtime, a smaller batch size may be chosen, but, not a larger one.

* __minimum_segment_size__: This parameter determines the minimum number of TensorFlow nodes in a TF-TRT engine, which means the TensorFlow subgraphs that have fewer nodes than this number will not be converted to TensorRT. Therefore, in general, smaller numbers such as 5 are preferred. This can also be used to change the minimum number of nodes in the optimized INT8 engines to change the final optimized graph to fine tune result accuracy.

* __max_workspace_size_bytes__: TF-TRT operators often require temporary workspace. This parameter limits the maximum size that any layer in the network can use. If insufficient scratch is provided, it is possible that TF-TRT may not be able to find an implementation for a given layer.

For more information, please refer to the [TF-TRT User Guide](https://docs.nvidia.com/deeplearning/frameworks/tf-trt-user-guide/).

## Next

In the next notebook we will demonstrate a TF-TRT conversion using Float32 precision.